In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
ACCESS_KEY = "bvdzdoz57qif0GYdiShG"
SECRET_KEY = "n5m5SjzQctzEMlPI9nMVPJgUM3BlQF8Otrp1jFg5"
MINIO_URL = "http://minio:9000"

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("HW2") \
    .config("spark.sql.adaptive.enabled", False) \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.sql.sources.bucketing.enabled", True) \
    .config("spark.executor.memory", "1000M") \
    .config("spark.driver.memory", "600M") \
    .config('spark.jars.packages', 
        "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-pom:1.12.365,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1"
    ) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_URL) \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-pom added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ee6169c4-274f-4e4e-a531-15346832bc49;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk-pom;1.12.365 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#ha

In [3]:
spark

# Задание 1

## Входные данные 
- Файл с данными по оттоку телеком оператора в США (churn.csv)
- Справочник с названиями штатов (state.json)
- Справочник с численностью населения территорий (определяется полем area code) внутри штатов (state.json)
- Террия с численностью населения меньше 10_000 считается **мелкой**

## Что нужно сделать
1. Посчитать количество отточных и неотточных абонентов (поле churn), исключив **мелкие** территории
2. Отчет должен быть выполнен в разрезе **каждого штата** с его полным наименованием
3. Описать возникающие узкие места при выполнении данной операции
4. Применить один из способов оптимизации для ускорения выполнения запроса (при допущении, что справочник численности населения **сильно меньше** основных данных)
5. Если существует еще какой-то способ, применить также и его отдельно от п.4 (при допущении, что справочник численности населения **сопоставим по размеру** с основными данными)
6. Кратко описать реализованные способы и в чем их практическая польза

- P.S. Одним из выбранных способов должен быть Bucket specific join
- P.P.S. При обосновании предлагаем прикладывать запуска команды df.explain()

In [4]:
churn_df = spark.read.option("header", True).csv("s3a://input/data/churn.csv")
state_dict = spark.read.json("s3a://input/data/state.json")
pop_dict = spark.read.json("s3a://input/data/population.json")

24/07/11 17:45:24 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


### Решение

Проведем сначала небольшой разведочный анализ (EDA) для понимания того, что хранится в каждом нашем датафрейме spark.

In [5]:
churn_df.show(5)

+---+-----+--------------+---------+------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+----------------------+-----+---+
| id|state|account length|area code|phone number|international plan|voice mail plan|number vmail messages|total day minutes|total day calls|total day charge|total eve minutes|total eve calls|total eve charge|total night minutes|total night calls|total night charge|total intl minutes|total intl calls|total intl charge|customer service calls|churn|val|
+---+-----+--------------+---------+------------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------

In [6]:
churn_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- state: string (nullable = true)
 |-- account length: string (nullable = true)
 |-- area code: string (nullable = true)
 |-- phone number: string (nullable = true)
 |-- international plan: string (nullable = true)
 |-- voice mail plan: string (nullable = true)
 |-- number vmail messages: string (nullable = true)
 |-- total day minutes: string (nullable = true)
 |-- total day calls: string (nullable = true)
 |-- total day charge: string (nullable = true)
 |-- total eve minutes: string (nullable = true)
 |-- total eve calls: string (nullable = true)
 |-- total eve charge: string (nullable = true)
 |-- total night minutes: string (nullable = true)
 |-- total night calls: string (nullable = true)
 |-- total night charge: string (nullable = true)
 |-- total intl minutes: string (nullable = true)
 |-- total intl calls: string (nullable = true)
 |-- total intl charge: string (nullable = true)
 |-- customer service calls: string (nullable = true)
 |--

In [7]:
state_dict.show(5)

+--------+----------+
|state_id|state_name|
+--------+----------+
|      AL|   Alabama|
|      AK|    Alaska|
|      AZ|   Arizona|
|      AR|  Arkansas|
|      CA|California|
+--------+----------+
only showing top 5 rows



In [8]:
state_dict.printSchema()

root
 |-- state_id: string (nullable = true)
 |-- state_name: string (nullable = true)



In [9]:
pop_dict.show(5)

+---------+----------+
|area code|population|
+---------+----------+
|      131|     15742|
|      139|     20233|
|      145|     54682|
|      153|     34569|
|      155|     58342|
+---------+----------+
only showing top 5 rows



In [10]:
pop_dict.printSchema()

root
 |-- area code: long (nullable = true)
 |-- population: long (nullable = true)



In [11]:
#зафиксируем трешхолд для последующей оптимизации
POP_THRESHOLD = 10_000

In [12]:
#уберем мелкие территории, так называемый метод Predicate pushdown перенесли шаг фильтрации ближе к источнику
churn_and_pop = churn_df.join(pop_dict, on="area code", how="inner") \
    .filter(F.col("population") > POP_THRESHOLD)

In [13]:
result = churn_and_pop.join(state_dict, churn_and_pop.state==state_dict.state_id, how="left") \
    .groupBy("state_name") \
    .pivot("churn") \
    .agg(F.count("*")) \
    .orderBy("state_name")

In [14]:
result.explain()
result.show(truncate=False)

== Physical Plan ==
*(12) Sort [state_name#72 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(state_name#72 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=343]
   +- *(11) Project [state_name#72, __pivot_count(1) AS `count(1)`#345[0] AS False#346L, __pivot_count(1) AS `count(1)`#345[1] AS True#347L]
      +- HashAggregate(keys=[state_name#72], functions=[pivotfirst(churn#38, count(1)#339L, False, True, 0, 0)])
         +- Exchange hashpartitioning(state_name#72, 200), ENSURE_REQUIREMENTS, [plan_id=338]
            +- HashAggregate(keys=[state_name#72], functions=[partial_pivotfirst(churn#38, count(1)#339L, False, True, 0, 0)])
               +- *(10) HashAggregate(keys=[state_name#72, churn#38], functions=[count(1)])
                  +- Exchange hashpartitioning(state_name#72, churn#38, 200), ENSURE_REQUIREMENTS, [plan_id=333]
                     +- *(9) HashAggregate(keys=[state_name#72, churn#38], functions=[partial_count(1)])
                        +- *(9) Project [

+--------------------+-----+----+
|state_name          |False|True|
+--------------------+-----+----+
|Alabama             |58   |7   |
|Alaska              |44   |3   |
|Arizona             |53   |4   |
|Arkansas            |35   |9   |
|California          |21   |8   |
|Colorado            |49   |7   |
|Connecticut         |56   |8   |
|Delaware            |48   |8   |
|District of Columbia|43   |4   |
|Florida             |45   |8   |
|Georgia             |42   |8   |
|Hawaii              |42   |3   |
|Idaho               |53   |9   |
|Illinois            |49   |5   |
|Indiana             |59   |9   |
|Iowa                |34   |3   |
|Kansas              |50   |12  |
|Kentucky            |51   |8   |
|Louisiana           |47   |4   |
|Maine               |49   |13  |
+--------------------+-----+----+
only showing top 20 rows



Следует перечислить следующие узкие места:
* т.к. наши файлы pop_dict и state_dict имеют размерность точно меньше <10МБ , то при соединении мы можем использовать BroadcastJoinThreshold было бы тут эффективнее
* из прочего пункта поскольку у нас sort merge join, то по плану видно, что имеется четыре операции перемешивания exchange до операции merge(логично было бы что два поскольку два join, поэтому вопросики к catalyst в данном случае либо ко мне) 
* сильный перекос в timeline в spark ui stages

### Оптимизация 1 - broadcast has join

In [15]:
pop_dict_broad = F.broadcast(pop_dict)
state_dict_broad = F.broadcast(state_dict)

In [16]:
churn_and_pop_broad = churn_df.join(pop_dict_broad, on="area code", how="inner") \
    .filter(F.col("population") > POP_THRESHOLD)

In [17]:
result_broad = churn_and_pop_broad.join(state_dict_broad, churn_and_pop_broad.state==state_dict.state_id, how="left") \
    .groupBy("state_name") \
    .pivot("churn") \
    .agg(F.count("*")) \
    .orderBy("state_name")

In [18]:
result_broad.explain()
result_broad.show(truncate=False)

== Physical Plan ==
*(6) Sort [state_name#72 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(state_name#72 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=862]
   +- *(5) Project [state_name#72, __pivot_count(1) AS `count(1)`#518[0] AS False#519L, __pivot_count(1) AS `count(1)`#518[1] AS True#520L]
      +- HashAggregate(keys=[state_name#72], functions=[pivotfirst(churn#38, count(1)#512L, False, True, 0, 0)])
         +- Exchange hashpartitioning(state_name#72, 200), ENSURE_REQUIREMENTS, [plan_id=857]
            +- HashAggregate(keys=[state_name#72], functions=[partial_pivotfirst(churn#38, count(1)#512L, False, True, 0, 0)])
               +- *(4) HashAggregate(keys=[state_name#72, churn#38], functions=[count(1)])
                  +- Exchange hashpartitioning(state_name#72, churn#38, 200), ENSURE_REQUIREMENTS, [plan_id=852]
                     +- *(3) HashAggregate(keys=[state_name#72, churn#38], functions=[partial_count(1)])
                        +- *(3) Project [chu

+--------------------+-----+----+
|state_name          |False|True|
+--------------------+-----+----+
|Alabama             |58   |7   |
|Alaska              |44   |3   |
|Arizona             |53   |4   |
|Arkansas            |35   |9   |
|California          |21   |8   |
|Colorado            |49   |7   |
|Connecticut         |56   |8   |
|Delaware            |48   |8   |
|District of Columbia|43   |4   |
|Florida             |45   |8   |
|Georgia             |42   |8   |
|Hawaii              |42   |3   |
|Idaho               |53   |9   |
|Illinois            |49   |5   |
|Indiana             |59   |9   |
|Iowa                |34   |3   |
|Kansas              |50   |12  |
|Kentucky            |51   |8   |
|Louisiana           |47   |4   |
|Maine               |49   |13  |
+--------------------+-----+----+
only showing top 20 rows



Broadcast hash join дает преимущество в виде скорости за счет того, что данные содержит локально на каждом executor и нет необходимости производить сетевую операцию - shuffle

### Оптимизация 2 - bucket specific join

In [19]:
#бакитируем один из датафреймов чтобы репартирование при join происходило без shuffle и sort
churn_df.repartition(1) \
    .write \
    .mode("overwrite") \
    .bucketBy(100, col="area code") \
    .option("path", "s3a://input/data/bucket/churn_bucket") \
    .saveAsTable("churn_bucket")

In [20]:
pop_dict \
    .withColumn("area code", F.col("area code").cast("string")) \
    .repartition(1) \
    .write \
    .bucketBy(100, "area code") \
    .option("path", "s3a://input/data/bucket/pop_bucket") \
    .saveAsTable("pop_bucket")

In [21]:
churn_bucket = spark.table("churn_bucket")
pop_bucket = spark.table("pop_bucket")

In [22]:
churn_and_pop_bucket = churn_bucket.join(pop_bucket, on="area code", how="inner") \
    .filter(F.col("population") > POP_THRESHOLD)

In [23]:
result_bucket = churn_and_pop_bucket.join(state_dict, churn_and_pop_bucket.state==state_dict.state_id, how="left") \
    .groupBy("state_name") \
    .pivot("churn") \
    .agg(F.count("*")) \
    .orderBy("state_name")

In [24]:
result_bucket.explain()
result_bucket.show(truncate=False)

== Physical Plan ==
*(10) Sort [state_name#72 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(state_name#72 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=1445]
   +- *(9) Project [state_name#72, __pivot_count(1) AS `count(1)`#792[0] AS False#793L, __pivot_count(1) AS `count(1)`#792[1] AS True#794L]
      +- HashAggregate(keys=[state_name#72], functions=[pivotfirst(churn#648, count(1)#786L, False, True, 0, 0)])
         +- Exchange hashpartitioning(state_name#72, 200), ENSURE_REQUIREMENTS, [plan_id=1440]
            +- HashAggregate(keys=[state_name#72], functions=[partial_pivotfirst(churn#648, count(1)#786L, False, True, 0, 0)])
               +- *(8) HashAggregate(keys=[state_name#72, churn#648], functions=[count(1)])
                  +- Exchange hashpartitioning(state_name#72, churn#648, 200), ENSURE_REQUIREMENTS, [plan_id=1435]
                     +- *(7) HashAggregate(keys=[state_name#72, churn#648], functions=[partial_count(1)])
                        +- *(7) Pro

+--------------------+-----+----+
|state_name          |False|True|
+--------------------+-----+----+
|Alabama             |58   |7   |
|Alaska              |44   |3   |
|Arizona             |53   |4   |
|Arkansas            |35   |9   |
|California          |21   |8   |
|Colorado            |49   |7   |
|Connecticut         |56   |8   |
|Delaware            |48   |8   |
|District of Columbia|43   |4   |
|Florida             |45   |8   |
|Georgia             |42   |8   |
|Hawaii              |42   |3   |
|Idaho               |53   |9   |
|Illinois            |49   |5   |
|Indiana             |59   |9   |
|Iowa                |34   |3   |
|Kansas              |50   |12  |
|Kentucky            |51   |8   |
|Louisiana           |47   |4   |
|Maine               |49   |13  |
+--------------------+-----+----+
only showing top 20 rows



Видим по плану запроса, что у нас отсутвует shuffle по area code для таблиц pop_bucket и churn_bucket, тем самым мы получили прирост производительсности по времени.

# Задание 2

## Входные данные 

*skew_transactions.csv* - информация о длительности просомтра контента пользователям
колонки:
1. user_uid — уникальный идентификатор пользователя
2. element_uid — уникальный идентификатор контента
3. watched_time — время просмотра в секундах

*catalogue.json* - каталог с описанием контента и метаинформации по нему
колонки:
1. type — тип элемента
2. duration — длительность в минутах (средняя длительность эпизода в случае с сериалами и многосерийными фильмами), округлённая до десятков
3. attributes — анонимизированные атрибуты данного элемента
4. availability — доступные права на элемент(subscription, purchase, rent)
5. feature_1 — анонимизированная вещественная переменная
6. feature_2 — анонимизированная вещественная переменная
7. feature_3 — анонимизированная порядковая переменная
8. feature_4 — анонимизированная вещественная переменная
9. feature_5 — анонимизированная вещественная переменная

## Что нужно сделать
1. Выполните join основных данных со справочником используя DataFrame API (по колонке id для контента - `element_uid`)
2. Описать проблему в датасетах с точки зрения обработки Spark
3. Решить задачу любым способом
4. Решить задачу с помощью salt-join подхода

P.S. Как вы можете заметить при просмотре данных по пользователями, нужный нам ключ для операции будет перекошен (90% строк представлены на фильм, очень популярный среди смотревших) - это нужно доказать в рамках п.2

### Решение 

In [25]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd

In [27]:
skew_df = spark.read.option("header", True).csv("s3a://input/data/skew_transactions.csv")

In [28]:
catalogue_df = pd.read_json("datasets/catalogue.json", dtype=str)

In [29]:
catalogue_df.head(5)

,1983,3783,5208,9744,1912,7944,10021,2166,6833,7904,...,7971,4178,9288,8756,598,6643,166,9242,6427,8452
type,movie,movie,movie,movie,movie,movie,movie,movie,movie,movie,...,series,series,series,series,series,series,series,series,series,series
availability,"['purchase', 'rent', 'subscription']","['purchase', 'rent', 'subscription']","['purchase', 'rent', 'subscription']","['purchase', 'rent', 'subscription']","['purchase', 'rent']","['purchase', 'rent', 'subscription']",[],[],[],"['purchase', 'rent', 'subscription']",...,['purchase'],['subscription'],['subscription'],['subscription'],['subscription'],"['purchase', 'subscription']",['subscription'],['purchase'],['subscription'],['subscription']
duration,140,110,90,120,110,110,110,90,90,120,...,20,30,30,60,50,50,30,40,30,40
feature_1,1657223.396513469,35565207.694893226,13270676.52431015,21749917.409822803,9212963.985682394,13214708.075501492,9045058.639256418,14918550.088817373,5589006.925321744,32362003.812979225,...,36693573.4048775,37810096.832399,38399364.92104859,41914615.73942064,43126613.14615511,42570403.52787781,43067650.320556894,25423401.689006824,13452573.982938292,38450199.365910105
feature_2,0.7536096584,0.7662537759,0.7654246597000001,0.7578744427,0.7595661572,0.7536096584,0.7378191549,0.7044612684,0.7510260541,0.7029805793,...,0.7863780695,0.7536096584,0.7492931785,0.74050001,0.74050001,0.7662537759,0.6929487826,0.6514504811,0.7662537759,0.6929487826


In [30]:
#транспонируем датафрейм и также индекс обозначим как столбец element_uid

catalogue_df = catalogue_df.transpose().reset_index()
catalogue_df = catalogue_df.rename(columns={"index": "element_uid"})


In [31]:
catalogue_df.head(5)

,element_uid,type,availability,duration,feature_1,feature_2,feature_3,feature_4,feature_5,attributes
0,1983,movie,"['purchase', 'rent', 'subscription']",140,1657223.396513469,0.7536096584,39,1.1194091265,0.0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,3783,movie,"['purchase', 'rent', 'subscription']",110,35565207.694893226,0.7662537759,41,1.1386044027,0.6547073468,"[1, 26, 27, 28, 29, 7, 30, 31, 32, 10, 14, 15,..."
2,5208,movie,"['purchase', 'rent', 'subscription']",90,13270676.52431015,0.7654246597000001,27,1.1318073548,0.5927161087,"[1, 38, 39, 40, 7, 41, 42, 43, 14, 15, 17, 18,..."
3,9744,movie,"['purchase', 'rent', 'subscription']",120,21749917.409822803,0.7578744427,26,1.1335254517,0.6547073468,"[1, 47, 48, 49, 50, 51, 52, 53, 32, 42, 54, 14..."
4,1912,movie,"['purchase', 'rent']",110,9212963.985682394,0.7595661572,7,1.1101274384,0.6547073468,"[1, 59, 60, 61, 62, 7, 52, 63, 10, 42, 54, 17,..."


In [32]:
#посмотрим на уникальные значения
catalogue_df["element_uid"].value_counts(ascending=False)

element_uid
1983     1
10053    1
2784     1
6768     1
4798     1
        ..
248      1
5937     1
6737     1
8970     1
8452     1
Name: count, Length: 10200, dtype: int64

In [33]:
catalogue_df = spark.createDataFrame(catalogue_df)

In [34]:
skew_df.show(5)

+-----------+--------+------------+
|element_uid|user_uid|watched_time|
+-----------+--------+------------+
|       6130|  563180|        3264|
|       2771|  133315|       10557|
|        984|  290104|        6845|
|       6980|  385385|        2734|
|       2714|  408794|        7583|
+-----------+--------+------------+
only showing top 5 rows



In [35]:
#также посмотрим на униклаьные значения element_uid, чтобы увидеть что распределение столбца element_uid перекошено
skew_df.groupBy("element_uid").count().orderBy(F.col("count").desc()).show(5)

+-----------+-------+
|element_uid|  count|
+-----------+-------+
|       2714|2732800|
|        747|  61272|
|      10170|    299|
|       9898|    299|
|       9837|    299|
+-----------+-------+
only showing top 5 rows



In [36]:
result = skew_df.join(catalogue_df.select("element_uid", "type", "availability", "duration"), on="element_uid", how="left")

In [37]:
result.explain()
result.show(truncate=False)

== Physical Plan ==
*(4) Project [element_uid#860, user_uid#861, watched_time#862, type#867, availability#868, duration#869]
+- *(4) SortMergeJoin [cast(element_uid#860 as bigint)], [element_uid#866L], LeftOuter
   :- *(1) Sort [cast(element_uid#860 as bigint) ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(cast(element_uid#860 as bigint), 200), ENSURE_REQUIREMENTS, [plan_id=1823]
   :     +- FileScan csv [element_uid#860,user_uid#861,watched_time#862] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[s3a://input/data/skew_transactions.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<element_uid:string,user_uid:string,watched_time:string>
   +- *(3) Sort [element_uid#866L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(element_uid#866L, 200), ENSURE_REQUIREMENTS, [plan_id=1834]
         +- *(2) Project [element_uid#866L, type#867, availability#868, duration#869]
            +- *(2) Filter isnotnull(element_ui

24/07/11 17:46:05 WARN TaskSetManager: Stage 44 contains a task of very large size (1035 KiB). The maximum recommended task size is 1000 KiB.


+-----------+--------+------------+-----+------------------------------------+--------+
|element_uid|user_uid|watched_time|type |availability                        |duration|
+-----------+--------+------------+-----+------------------------------------+--------+
|6130       |563180  |3264        |movie|['purchase', 'rent']                |90      |
|2714       |408794  |7583        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |247546  |4450        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |43556   |7569        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |169048  |7577        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |542158  |8834        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |356858  |7763        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |277533  |4781        |movie|['purchase', 'rent', 'subscription']|130     |
|2714       |345703  |7639      

Как видно по плану у нас выполняется дефолтный sort merge join, мы имеем shuffle, а с учетом перекоса ключа есть следующие проблемы:
* **Излишняя нагрузка на некоторые узлы**: В sort-merge join данные должны быть отсортированы и объединены по ключам. Если ключи распределены неравномерно, узлы, отвечающие за частые ключи, получат больше данных для обработки. Это приводит к дисбалансу нагрузки(судя по spark ui timeline), когда некоторые узлы будут перегружены, становясь узкими местами (bottlenecks), а другие будут простаивать.
* **Использование памяти в режиме memory spill**: Не в данном кейсе, но в целом, перегруженные узлы могут исчерпать доступную память из-за большого объема данных, что может привести к сбоям или необходимости использования дискового пространства для выполнения операций, что замедляет процесс.
* **Большое количество операций shuffle**: Операция происходит между узлами, что прямо сказывается на производительсности и скорости выполенения

### Решение с оптимизацией - broadcast hash join
Опять же поскольку одна из таблиц, а конкретнее catalogue_df у нас 2.7Mb < 10Mb (смотрел в хранилище minio, не отрицаю возможно есть более релевантный способ), то мы можем использовать broadcast hash join чтобы избежать лишнего shuffle и следовательно лишнего сетевого взаимодействия

In [38]:
catalogue_broad = F.broadcast(catalogue_df)
skew_repartition = skew_df.repartition(5)

In [39]:
result_broad = skew_repartition.join(catalogue_broad.select("element_uid", "type", "availability", "duration"), on="element_uid", how="left")

In [40]:
result_broad.explain()
result_broad.show(truncate=False)

== Physical Plan ==
*(2) Project [element_uid#860, user_uid#861, watched_time#862, type#867, availability#868, duration#869]
+- *(2) BroadcastHashJoin [cast(element_uid#860 as bigint)], [element_uid#866L], LeftOuter, BuildRight, false
   :- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=1944]
   :  +- FileScan csv [element_uid#860,user_uid#861,watched_time#862] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[s3a://input/data/skew_transactions.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<element_uid:string,user_uid:string,watched_time:string>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, true]),false), [plan_id=1955]
      +- *(1) Project [element_uid#866L, type#867, availability#868, duration#869]
         +- *(1) Filter isnotnull(element_uid#866L)
            +- *(1) Scan ExistingRDD[element_uid#866L,type#867,availability#868,duration#869,feature_1#870,feature_2#871,feature_3#872,feat

24/07/11 17:46:07 WARN TaskSetManager: Stage 55 contains a task of very large size (1035 KiB). The maximum recommended task size is 1000 KiB.


+-----------+--------+------------+---------------+------------------------------------+--------+
|element_uid|user_uid|watched_time|type           |availability                        |duration|
+-----------+--------+------------+---------------+------------------------------------+--------+
|528        |277909  |1263        |movie          |['purchase', 'rent', 'subscription']|120     |
|2714       |482595  |7733        |movie          |['purchase', 'rent', 'subscription']|130     |
|2714       |482595  |7733        |movie          |['purchase', 'rent', 'subscription']|130     |
|2714       |482595  |7733        |movie          |['purchase', 'rent', 'subscription']|130     |
|2714       |482595  |7733        |movie          |['purchase', 'rent', 'subscription']|130     |
|7594       |442511  |2838        |movie          |['purchase', 'rent', 'subscription']|60      |
|2714       |457718  |841         |movie          |['purchase', 'rent', 'subscription']|130     |
|2714       |457718 

Мы получили прирост производительности:
* было - 2с
* стало - 0.5с
Конечно кидает warning, но в данной задаче он прям не критичен. Мы избегаем этап shuffle.

### Решение с оптимизацией - salting

In [41]:
def data_skew_helper(left, right, key, number_of_partitions, how="inner"):
    #generate random salt for left dataframe
    salt_value = F.lit(F.rand() * number_of_partitions % number_of_partitions).cast("int")
    left = left.withColumn("salt", salt_value)

    #Creating a column with an array of all possible salt values for right dataframe
    salt_col = F.explode(F.array([F.lit(i) for i in range(number_of_partitions)])).alias("salt")
    right = right.select("*",  salt_col)

    return left.join(right, [key, "salt"], how)

In [42]:
result_salt = data_skew_helper(skew_df, catalogue_df.select("element_uid", "type", "availability", "duration"), "element_uid", 5, how="left")

In [43]:
result_salt.explain()
result_salt.show(truncate=False)

== Physical Plan ==
*(5) Project [element_uid#860, salt#1002, user_uid#861, watched_time#862, type#867, availability#868, duration#869]
+- *(5) SortMergeJoin [cast(element_uid#860 as bigint), salt#1002], [element_uid#866L, salt#1008], LeftOuter
   :- *(2) Sort [cast(element_uid#860 as bigint) ASC NULLS FIRST, salt#1002 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(cast(element_uid#860 as bigint), salt#1002, 200), ENSURE_REQUIREMENTS, [plan_id=2064]
   :     +- *(1) Project [element_uid#860, user_uid#861, watched_time#862, cast(((rand(4144267069676422802) * 5.0) % 5.0) as int) AS salt#1002]
   :        +- FileScan csv [element_uid#860,user_uid#861,watched_time#862] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[s3a://input/data/skew_transactions.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<element_uid:string,user_uid:string,watched_time:string>
   +- *(4) Sort [element_uid#866L ASC NULLS FIRST, salt#1008 ASC NULLS 

24/07/11 17:46:10 WARN TaskSetManager: Stage 58 contains a task of very large size (1035 KiB). The maximum recommended task size is 1000 KiB.


+-----------+----+--------+------------+---------------+------------------------------------+--------+
|element_uid|salt|user_uid|watched_time|type           |availability                        |duration|
+-----------+----+--------+------------+---------------+------------------------------------+--------+
|2692       |1   |167038  |0           |multipart_movie|['purchase', 'rent', 'subscription']|60      |
|2714       |2   |408794  |7583        |movie          |['purchase', 'rent', 'subscription']|130     |
|2714       |2   |169048  |7577        |movie          |['purchase', 'rent', 'subscription']|130     |
|2714       |2   |356858  |7763        |movie          |['purchase', 'rent', 'subscription']|130     |
|2714       |2   |345703  |7639        |movie          |['purchase', 'rent', 'subscription']|130     |
|2714       |2   |343066  |268         |movie          |['purchase', 'rent', 'subscription']|130     |
|2714       |2   |57572   |7616        |movie          |['purchase', 'ren

In [44]:
#чтобы убедиться что распределение по партициям у нас лучше, чем при простом merge sort join без salt
result_salt_check_partition = result_salt.groupBy("element_uid", "salt").count().orderBy(F.col("count").desc())

In [45]:
result_salt_check_partition.show(truncate=False)

24/07/11 17:46:11 WARN TaskSetManager: Stage 73 contains a task of very large size (1035 KiB). The maximum recommended task size is 1000 KiB.


+-----------+----+------+
|element_uid|salt|count |
+-----------+----+------+
|2714       |0   |546878|
|2714       |1   |546700|
|2714       |2   |546681|
|2714       |3   |546279|
|2714       |4   |546262|
|747        |1   |12522 |
|747        |3   |12335 |
|747        |4   |12194 |
|747        |2   |12151 |
|747        |0   |12070 |
|4401       |2   |78    |
|8763       |2   |78    |
|7923       |2   |78    |
|9766       |4   |77    |
|3332       |2   |77    |
|4197       |0   |76    |
|3162       |3   |75    |
|8804       |1   |74    |
|6306       |1   |73    |
|5480       |4   |73    |
+-----------+----+------+
only showing top 20 rows



В данном случае мы получили время выполнения 0.6с, что быстрее по сравнению с раннее двумя использованными способами. Это происходит за счет того, что происходит смягчение перекоса данных, что позволяет равномерно распределить данные по нескольким ключам, тем самым помогая сбалансировать нагрузку между узлами. Но как видим по плану работ shuffle при этом у нас сохраняется как при простом sort merge join.

# Задание 3

## Входные данные 

*cut_transactions.csv*  — информация о длительности просомтра контента пользователям

Описание фичей в cut_transactions.csv: 
1. user_uid — уникальный идентификатор пользователя
2.  element_uid — уникальный идентификатор контента
3.  watched_time — время просмотра в секундах

*cut_ratings.csv*  — информация об оценках, поставленных пользователями

Описание фичей в cut_ratings.csv: 
1. user_uid — уникальный идентификатор пользователя 
2. element_uid — уникальный идентификатор контента 
3. rating — поставленный пользователем рейтинг

*ids.csv*  — выборка пользователей
Описание фичей в ids.csv: 
1. user_uid — уникальный идентификатор пользователя 


## Что нужно сделать
Для каждого пользователя из выборки посчитать:
1. Максимальное и минимальное время просмотра фильмов с оценками 8, 9 и 10 
2. Название фичи должно быть в формате feat_агрегирующая_функция_watched_time_rating_оценка. 
3. Если у пользователь не ставил оценки 8, 9 и 10 то значение фичей должно быть null
4. Описать принятые при разработки кода решения и возможные оптимизации

P.S. На каждом этапе обработки должно быть должны агрегироваться минимально возможные объемы данных (сокращаем затраты на shuflle)

In [46]:
transactions_df = spark.read.option("header", True).csv("s3a://input/data/cut_transactions.csv")
ratings_df = spark.read.option("header", True).csv("s3a://input/data/cut_ratings.csv")
ratings_df = F.broadcast(ratings_df)
sample = spark.read.option("header", True).csv("s3a://input/data/ids.csv")
sample = F.broadcast(sample)

In [47]:
transactions_df.printSchema()

root
 |-- element_uid: string (nullable = true)
 |-- user_uid: string (nullable = true)
 |-- watched_time: string (nullable = true)



In [48]:
transactions_df.show(5)

+-----------+--------+------------+
|element_uid|user_uid|watched_time|
+-----------+--------+------------+
|       6130|  563180|        3264|
|       2771|  133315|       10557|
|        984|  290104|        6845|
|       6980|  385385|        2734|
|       2714|  408794|        7583|
+-----------+--------+------------+
only showing top 5 rows



In [49]:
ratings_df.printSchema()

root
 |-- user_uid: string (nullable = true)
 |-- element_uid: string (nullable = true)
 |-- rating: string (nullable = true)



In [50]:
ratings_df.show(5)

+--------+-----------+------+
|user_uid|element_uid|rating|
+--------+-----------+------+
|  566386|        747|     9|
|   61690|       1415|     8|
|  153993|       1912|    10|
|  207861|       2714|    10|
|  526934|       4558|     9|
+--------+-----------+------+
only showing top 5 rows



24/07/11 17:46:15 WARN HintErrorLogger: A join hint (strategy=broadcast) is specified but it is not part of a join relation.


In [51]:
sample.printSchema()

root
 |-- user_uid: string (nullable = true)



In [52]:
sample.show(5)

+--------+
|user_uid|
+--------+
|  207861|
|  355521|
|   74017|
|  394262|
|  524044|
+--------+
only showing top 5 rows



24/07/11 17:46:15 WARN HintErrorLogger: A join hint (strategy=broadcast) is specified but it is not part of a join relation.


### Решение

In [53]:
full_table = transactions_df.join(ratings_df , on=["user_uid", "element_uid"], how="inner").join(sample, on="user_uid", how="inner")

In [54]:
full_table.show(5)

+--------+-----------+------------+------+
|user_uid|element_uid|watched_time|rating|
+--------+-----------+------------+------+
|  355521|        747|        7210|    10|
|  207861|       2714|        8088|    10|
|  394262|       7143|        5717|    10|
|   74017|       1048|        6001|    10|
|  145083|       6138|        7361|    10|
+--------+-----------+------------+------+
only showing top 5 rows



**Непосредственно решение нашей задачи**

In [55]:
filtered_table = full_table.filter(F.col("rating").isin([8, 9, 10]))
full_table_filter = filtered_table.groupBy("user_uid", "rating") \
                                  .agg(F.min("watched_time").alias("min_watched_time"), 
                                       F.max("watched_time").alias("max_watched_time"))


In [56]:
# fulfilling the requirement of column format 
ratings = [8, 9, 10]
result_df = sample

for rating in ratings:
    rating_df = full_table_filter.filter(F.col("rating") == rating) \
                                 .select(
                                     F.col("user_uid"),
                                     F.col("min_watched_time").alias(f"feat_min_watched_time_rating_{rating}"),
                                     F.col("max_watched_time").alias(f"feat_max_watched_time_rating_{rating}")
                                 )
    result_df = result_df.join(rating_df, on="user_uid", how="left")

# Sort table
result_table = result_df.orderBy("user_uid")

In [57]:
result_table.show(10)

24/07/11 17:46:15 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for left outer join.


+--------+------------------------------+------------------------------+------------------------------+------------------------------+-------------------------------+-------------------------------+
|user_uid|feat_min_watched_time_rating_8|feat_max_watched_time_rating_8|feat_min_watched_time_rating_9|feat_max_watched_time_rating_9|feat_min_watched_time_rating_10|feat_max_watched_time_rating_10|
+--------+------------------------------+------------------------------+------------------------------+------------------------------+-------------------------------+-------------------------------+
|   11384|                          NULL|                          NULL|                          NULL|                          NULL|                           6797|                           7631|
|  126454|                          NULL|                          NULL|                          NULL|                          NULL|                          34076|                          34076|
|  13

In [58]:
result_table.explain()
result_table.show(truncate=False)

24/07/11 17:46:18 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for left outer join.


== Physical Plan ==
*(20) Sort [user_uid#1144 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(user_uid#1144 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=3529]
   +- *(19) Project [user_uid#1144, feat_min_watched_time_rating_8#1231, feat_max_watched_time_rating_8#1232, feat_min_watched_time_rating_9#1241, feat_max_watched_time_rating_9#1242, feat_min_watched_time_rating_10#1261, feat_max_watched_time_rating_10#1262]
      +- *(19) SortMergeJoin [user_uid#1144], [user_uid#1267], LeftOuter
         :- *(13) Project [user_uid#1144, feat_min_watched_time_rating_8#1231, feat_max_watched_time_rating_8#1232, feat_min_watched_time_rating_9#1241, feat_max_watched_time_rating_9#1242]
         :  +- *(13) SortMergeJoin [user_uid#1144], [user_uid#1247], LeftOuter
         :     :- *(7) Project [user_uid#1144, feat_min_watched_time_rating_8#1231, feat_max_watched_time_rating_8#1232]
         :     :  +- *(7) SortMergeJoin [user_uid#1144], [user_uid#1099], LeftOuter
         :     :  

24/07/11 17:46:18 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for left outer join.


+--------+------------------------------+------------------------------+------------------------------+------------------------------+-------------------------------+-------------------------------+
|user_uid|feat_min_watched_time_rating_8|feat_max_watched_time_rating_8|feat_min_watched_time_rating_9|feat_max_watched_time_rating_9|feat_min_watched_time_rating_10|feat_max_watched_time_rating_10|
+--------+------------------------------+------------------------------+------------------------------+------------------------------+-------------------------------+-------------------------------+
|11384   |NULL                          |NULL                          |NULL                          |NULL                          |6797                           |7631                           |
|126454  |NULL                          |NULL                          |NULL                          |NULL                          |34076                          |34076                          |
|1316

Опять же использовали подход с использованием broadcast hash join, поскольку наша табличка sample и cut_ratings небольшая, то это позволяет избежать shuffle операций и значительно ускорить join операции.